In [ ]:
# [코드 3.1] requests 모듈 임포트 하기 (CH3. 데이터 수집하기.ipynb)

import requests

In [ ]:
# [코드 3.2] 네이버 메인 페이지로 요청 보내기 (CH3. 데이터 수집하기.ipynb)

naver_url = 'https://www.naver.com/'
naver_response = requests.get(naver_url)

In [ ]:
# [코드 3.3] 응답으로 받은 데이터에서 코드 찾기 (CH3. 데이터 수집하기.ipynb)

naver_response.text

In [ ]:
# [코드 3.4] 뷰티풀스프 모듈 임포트하고 뷰티풀스프 형 데이터 만들기 (CH3. 데이터 수집하기.ipynb)


import bs4

naver_bs = bs4.BeautifulSoup(naver_response.text, 'lxml')

In [ ]:
# [코드 3.5] find 메소드로 원하는 내용 찾기 (CH3. 데이터 수집하기.ipynb)

naver_bs.find('a', class_='issue')

In [ ]:
# [코드 3.6] find로 찾은 결과에서 내용만 보기 (CH3. 데이터 수집하기.ipynb)

result = naver_bs.find('a', class_='issue')
result.text

In [ ]:
# [코드 3.7] find_all 메소드로 원하는 내용 모두 찾기 (CH3. 데이터 수집하기.ipynb)

naver_bs.find_all('a', class_='issue')

In [ ]:
# [코드 3.8] find_all 결과에서 내용만 가져오기 (CH3. 데이터 수집하기.ipynb)

result_list = naver_bs.find_all('a', class_='issue')

for span in result_list:
    print(span.text)

In [ ]:
# [코드 3.9] 재무제표 페이지에서 테이블들 찾아오기 (CH3. 데이터 수집하기.ipynb)

import pandas as pd

fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701'
fs_page = requests.get(fs_url)
fs_tables = pd.read_html(fs_page.text)

In [ ]:
fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&gicode=A005930&cID=&MenuYn=Y&ReportGB=&NewMenuID=103&stkGb=701'
fs_page = requests.get(fs_url)
fs_tables = pd.read_html(fs_page.text)
fs_tables[0]

In [ ]:
# [코드 3.10] 포괄손익계산서(연간) 데이터프레임 가져오기 (CH3. 데이터 수집하기.ipynb)

fs_tables[0]

In [ ]:
# [코드 3.11] 포괄손익계산서(연간) 데이터프레임 인덱스와 컬럼 개선하기 (CH3. 데이터 수집하기.ipynb)

temp_df = fs_tables[0]
temp_df =  temp_df.set_index('IFRS(연결)')
temp_df = temp_df.iloc[:, :4] 

In [ ]:
# [코드 3.12] 포괄손익계산서(연간) 데이터프레임에서 사용할 행만 남기기 (CH3. 데이터 수집하기.ipynb)

temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

In [ ]:
temp_df

In [ ]:
# [코드 3.13] 재무상태표와 현금흐름표 데이터프레임 정리 (CH3. 데이터 수집하기.ipynb)

temp_df2 = fs_tables[2]
temp_df2 = temp_df2.set_index('IFRS(연결)')
temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

temp_df3 = fs_tables[4]
temp_df3 = temp_df3.set_index('IFRS(연결)')
temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

In [ ]:
# [코드 3.14] 재무제표 페이지의 데이터프레임들 이어 붙이기 (CH3. 데이터 수집하기.ipynb)

fs_df = pd.concat([temp_df, temp_df2, temp_df3])

In [ ]:
# [코드 3.15] 재무제표 데이터를 가져와 데이터프레임으로 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fs_dataframe(firm_code):
    fs_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Finance.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=103&stkGb=701&gicode=' + firm_code
    fs_page = requests.get(fs_url)
    fs_tables = pd.read_html(fs_page.text)

    # fs_bs = bs4.BeautifulSoup(fs_page.text, 'lxml')    
    # firm_name = fs_bs.find('h1', id='giName')
    temp_df = fs_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df[temp_df.columns[:4]]
    temp_df = temp_df.loc[['매출액', '영업이익', '당기순이익']]

    temp_df2 = fs_tables[2]
    temp_df2 = temp_df2.set_index(temp_df2.columns[0])
    temp_df2 = temp_df2.loc[['자산', '부채', '자본']]

    temp_df3 = fs_tables[4]
    temp_df3 = temp_df3.set_index(temp_df3.columns[0])
    temp_df3 = temp_df3.loc[['영업활동으로인한현금흐름']]

    fs_df = pd.concat([temp_df, temp_df2, temp_df3])
    
    return fs_df #, firm_name.text

In [ ]:
# [코드 3.16] make_fs_dataframe 함수를 이용하여 현대차 재무제표 데이터프레임 만들기 (CH3. 데이터 수집하기.ipynb)

test = make_fs_dataframe('A005380')

In [ ]:
test

In [ ]:
# [코드 3.17] 앞에서 완성한 데이터프레임 회사코드 넣어서 형태 바꾸기 (CH3. 데이터 수집하기.ipynb)

code = 'A005930'
fs_df = make_fs_dataframe(code)
col = fs_df.columns[0]
temp_df = pd.DataFrame({code : fs_df[col]})
temp_df = temp_df.T
temp_df.columns = [[col]*len(fs_df), temp_df.columns]

In [ ]:
temp_df

In [ ]:
# [코드 3.18] 데이터프레임 형태 바꾸기 완성 (CH3. 데이터 수집하기.ipynb)

code = 'A005930'
fs_df = make_fs_dataframe(code)

for num, col in enumerate(fs_df.columns):
    temp_df = pd.DataFrame({code : fs_df[col]})
    temp_df = temp_df.T
    temp_df.columns = [[col]*len(fs_df), temp_df.columns]
    if num == 0:
        total_df = temp_df
    else:
        total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)    

In [ ]:
total_df

In [ ]:
# [코드 3.19] 데이터프레임 형태 바꾸기 코드 함수화 (CH3. 데이터 수집하기.ipynb)

def change_df(firm_code, dataframe):
    for num, col in enumerate(dataframe.columns):
        temp_df = pd.DataFrame({firm_code : dataframe[col]})
        temp_df = temp_df.T
        temp_df.columns = [[col]*len(dataframe), temp_df.columns]
        if num == 0:
            total_df = temp_df
        else:
            total_df = pd.merge(total_df, temp_df, how='outer', left_index=True, right_index=True)    
    
    return total_df

In [ ]:
# [코드 3.20] 5개 회사의 재무제표 데이터 가져와서 합쳐보기 (CH3. 데이터 수집하기.ipynb)

firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']

for num, code in enumerate(firmcode_list):
    fs_df = make_fs_dataframe(code)
    fs_df_changed = change_df(code, fs_df)
    if num == 0 :
        total_fs = fs_df_changed
    else:
        total_fs = pd.concat([total_fs, fs_df_changed])

In [ ]:
total_fs

In [ ]:
# [코드 3.21] 재무 비율 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_fr_dataframe(firm_code):
    fr_url = 'https://comp.fnguide.com/SVO2/asp/SVD_FinanceRatio.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=104&stkGb=701&gicode=' + firm_code
    fr_page = requests.get(fr_url)
    fr_tables = pd.read_html(fr_page.text)
    # fr_bs = bs4.BeautifulSoup(fr_page.text, 'lxml')    
    # firm_name = fr_bs.find('h1', id='giName')
    temp_df = fr_tables[0]
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['유동비율계산에 참여한 계정 펼치기',
                           '부채비율계산에 참여한 계정 펼치기',
                           '영업이익률계산에 참여한 계정 펼치기',
                           'ROA계산에 참여한 계정 펼치기',
                           'ROIC계산에 참여한 계정 펼치기']]
    temp_df.index = ['유동비율', '부채비율', '영업이익률', 'ROA', 'ROIC']
    return temp_df#, firm_name.text

In [ ]:
make_fr_dataframe('A005930')

In [ ]:
# [코드 3.22] 5개 회사의 재무비율 데이터 가져와서 합쳐보기 (CH3. 데이터 수집하기.ipynb)

firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']

for num, code in enumerate(firmcode_list):
    fr_df = make_fr_dataframe(code)
    fr_df_changed = change_df(code, fr_df)
    if num == 0 :
        total_fr = fr_df_changed
    else:
        total_fr = pd.concat([total_fr, fr_df_changed])

In [ ]:
total_fr

In [ ]:
# [코드 3.23] 투자지표 데이터프레임을 만드는 함수 (CH3. 데이터 수집하기.ipynb)

def make_invest_dataframe(firm_code):
    invest_url = 'https://comp.fnguide.com/SVO2/asp/SVD_Invest.asp?pGB=1&cID=&MenuYn=Y&ReportGB=D&NewMenuID=105&stkGb=701&gicode=' + firm_code
    invest_page = requests.get(invest_url)
    invest_tables = pd.read_html(invest_page.text)
    # invest_bs = bs4.BeautifulSoup(invest_page.text, 'lxml')    
    # firm_name = invest_bs.find('h1', id='giName')
    temp_df = invest_tables[1]
    
    temp_df = temp_df.set_index(temp_df.columns[0])
    temp_df = temp_df.loc[['PER계산에 참여한 계정 펼치기',
                           'PCR계산에 참여한 계정 펼치기',
                           'PSR계산에 참여한 계정 펼치기',
                           'PBR계산에 참여한 계정 펼치기',
                          '총현금흐름']]
    temp_df.index = ['PER', 'PCR', 'PSR', 'PBR', '총현금흐름']
    return temp_df#, firm_name.text

In [ ]:
make_invest_dataframe('A005930')

In [ ]:
# [코드 3.24] 5개 회사의 투자지표 데이터 가져와서 합쳐보기 (CH3. 데이터 수집하기.ipynb)

firmcode_list = ['A005930', 'A005380', 'A035420', 'A003550', 'A034730']

for num, code in enumerate(firmcode_list):
    invest_df = make_invest_dataframe(code)
    invest_df_changed = change_df(code, invest_df)
    if num == 0 :
        total_invest = invest_df_changed 
    else:
        total_invest = pd.concat([total_invest, invest_df_changed])

In [ ]:
total_invest

In [ ]:
# [코드 3.25] 다운받은 종목 엑셀파일 읽어오기 (CH3. 데이터 수집하기.ipynb)

path = r'C:\quant_magic\주식 책 데이터\data.xls'
code_data = pd.read_excel(path)
code_data = code_data[['종목코드', '기업명']]

In [ ]:
# [코드 3.26] 종목코드 컬럼 개선 (CH3. 데이터 수집하기.ipynb)

def make_code(x):
    x = str(x)
    return 'A' + '0' * (6-len(x)) + x

code_data['종목코드'] = code_data['종목코드'].apply(make_code)

In [ ]:
# [코드 3.27] 모든 종목에 대해서 재무제표 데이터 가져오기 (CH3. 데이터 수집하기.ipynb)

import time

for num, code in enumerate(code_data['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            fs_df = make_fs_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            fs_df = make_fs_dataframe(code)
        fs_df_changed = change_df(code, fs_df)
        if num == 0 :
            total_fs = fs_df_changed
        else:
            total_fs = pd.concat([total_fs, fs_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

total_fs

In [ ]:
# [코드 3.28] 재무제표 데이터 엑셀로 저장 (CH3. 데이터 수집하기.ipynb)

total_fs.to_excel(r'C:\quant_magic\주식 책 데이터\재무제표데이터_220206.xlsx')

In [ ]:
# [코드 3.29] 모든 종목에 대해서 재무비율 데이터 가져오기 (CH3. 데이터 수집하기.ipynb)

for num, code in enumerate(code_data['종목코드']):
    try:
        print(num, code)
        time.sleep(1)
        try:
            fr_df = make_fr_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            fr_df = make_fr_dataframe(code)
        fr_df_changed = change_df(code, fr_df)
        if num == 0 :
            total_fr = fr_df_changed
        else:
            total_fr = pd.concat([total_fr, fr_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

In [ ]:
# [코드 3.30] 재무비율 데이터 엑셀로 저장 (CH3. 데이터 수집하기.ipynb)

total_fr.to_excel(r'C:\quant_magic\주식 책 데이터\재무비율데이터_220206.xlsx')

In [ ]:
# [코드 3.31] 모든 종목에 대해서 투자지표 데이터 가져오기 (CH3. 데이터 수집하기.ipynb)

for num, code in enumerate(code_data['종목코드'][313:]):
    try:
        print(num, code)
        time.sleep(1)
        try:
            invest_df = make_invest_dataframe(code)
        except requests.exceptions.Timeout:
            time.sleep(60)
            invest_df = make_invest_dataframe(code)
        invest_df_changed = change_df(code, invest_df)
        if num == 0 :
            total_invest = invest_df_changed
        else:
            total_invest = pd.concat([total_invest, invest_df_changed])
    except ValueError:
        continue
    except KeyError:
        continue

In [ ]:
# [코드 3.32] 투자지표 데이터 엑셀로 저장 (CH3. 데이터 수집하기.ipynb)

total_invest.to_excel(r'C:\quant_magic\주식 책 데이터\투자지표데이터_220206.xlsx')